In [1]:
import numpy as np

from pollen_manipulation.reachy_2.api import Reachy2ManipulationAPI

from pollen_vision.camera_wrappers.depthai import SDKWrapper
from pollen_vision.camera_wrappers.depthai.utils import get_config_file_path
from pollen_vision.perception import Perception

import FramesViewer.utils as fv_utils
from reachy2_sdk import ReachySDK

import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

T_world_cam = fv_utils.make_pose([0.052644, 0.01, 0.034150], [0, 0, 0])
T_world_cam[:3, :3] = np.array([[0, 0, 1], [-1, 0, 0], [0, -1, 0]])
T_world_cam = fv_utils.rotateInSelf(T_world_cam, [-48, 0, 0])

cam = SDKWrapper(get_config_file_path("CONFIG_SR"), compute_depth=True)

K = cam.get_K()

reachy = ReachySDK(host='localhost')
manipulation_api = Reachy2ManipulationAPI(reachy, T_world_cam, K)

perception = Perception(
    camera_wrapper=cam, T_world_cam=T_world_cam, freq=10.0
)
perception.start(visualize=False)

perception.set_tracked_objects(['bottle'])

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


[05/22/24 18:24:44] WARNING  Your inference package version 0.9.13 is out of date! Please upgrade to ]8;id=618361;file:///home/simsim/.virtualenvs/poc_llm_grasping/lib/python3.10/site-packages/inference/core/__init__.py\__init__.py]8;;\:]8;id=277977;file:///home/simsim/.virtualenvs/poc_llm_grasping/lib/python3.10/site-packages/inference/core/__init__.py#35\35]8;;\
                             version 0.11.0 of inference for the latest features and bug fixes by                  
                             running `pip install --upgrade inference`.                                            

[19443010C1D4782700] [3.1] [1.435] [ColorCamera(1)] [warning] Unsupported resolution set for detected camera OV9782, needs 800_P or 720_P. Defaulting to 800_P
[19443010C1D4782700] [3.1] [1.436] [ColorCamera(0)] [warning] Unsupported resolution set for detected camera OV9782, needs 800_P or 720_P. Defaulting to 800_P


Cameras not initialized.


model func:  <module 'contact_graspnet_pytorch.contact_graspnet' from '/home/simsim/.virtualenvs/poc_llm_grasping/lib/python3.10/site-packages/contact_graspnet_pytorch/contact_graspnet.py'>
/home/simsim/.cache/huggingface/hub/models--pollen-robotics--contact_graspnet/snapshots/16f1311d6380e2e6d4394d250897585fea0258a2/checkpoints/contact_graspnet/checkpoints/model.pt
=> Loading checkpoint from local file...
Adding tracking for object: bottle


In [2]:
manipulation_api.turn_robot_on()

In [4]:
manipulation_api.goto_rest_position(left=True)

In [5]:
object_info = [info for info in perception.get_objects_infos() if info['name'] == 'bottle']

if len(object_info) == 0:
    print('No bottle found')

elif len(object_info) == 1:
    bottle_info = object_info[0]
    print('Found a bottle')

else:
    print(f'Found {len(object_info)} bottles')

Found a bottle


In [22]:
from scipy.spatial.transform import Rotation as R

def compute_goal_diff(fk_matrix, goal_pose):
    return np.linalg.norm(fk_matrix - selected_array)

def compute_l2_distance(fk_matrix, goal_pose):
    return np.linalg.norm(fk_matrix[:-1, 3] - goal_pose[:-1, 3])


def produce_symetrical_poses(grasp_poses):
    symetrical_grasp_poses = []

    for grasp_pose in grasp_poses:
        symetrical_grasp_pose = np.eye(4)
        x, y, z = grasp_pose[:-1, 3]
        symetrical_grasp_pose[:-1, 3] = [x, -y, z]

        rotation_matrix = grasp_pose[:-1, :-1]
        roll, pitch, yaw = R.from_matrix(rotation_matrix).as_euler("xyz", degrees=True)
        symetrical_grasp_pose[:-1, :-1] = R.from_euler("xyz", [-roll, pitch, -yaw], degrees=True).as_matrix()
        symetrical_grasp_poses.append(symetrical_grasp_pose)

    return symetrical_grasp_poses

# Define the initial pose matrices for each arm used by goto_rest_position API method
right_start_pose = np.array([
    [0.0, 0.0, -1.0, 0.20],
    [0.0, 1.0, 0.0, -0.24],
    [1.0, 0.0, 0.0, -0.23],
    [0.0, 0.0, 0.0, 1.0],
])
left_start_pose = np.array([
    [0.0, 0.0, -1.0, 0.20],
    [0.0, 1.0, 0.0, 0.24],
    [1.0, 0.0, 0.0, -0.23],
    [0.0, 0.0, 0.0, 1.0],
])

In [24]:
import time

pose = bottle_info["pose"]
rgb = bottle_info["rgb"]
mask = bottle_info["mask"]
depth = bottle_info["depth"]

left = False

if left:
    arm = manipulation_api.reachy.l_arm
    symetrical_arm = manipulation_api.reachy.r_arm
    start_pose = left_start_pose
    
else:
    arm = manipulation_api.reachy.r_arm
    symetrical_arm = manipulation_api.reachy.l_arm
    start_pose = right_start_pose

grasp_poses, _ = manipulation_api.get_reachable_grasp_poses(rgb, depth, mask, left)
grasp_poses.insert(0, start_pose)

symetrical_grasp_poses = produce_symetrical_poses(grasp_poses)

print(len(grasp_poses))

Converting depth to point cloud(s)...
using local regions
Extracted Region Cube Size:  0.5699999332427979
Generated 98 grasps for object 1
Number of grasp poses: 196
Number of reachable grasp poses: 16
4


In [26]:
slider = widgets.IntSlider(
    value=0,
    min=0,
    max=len(grasp_pose) - 1,
    step=1,
    description='Tableau:',
    orientation='horizontal',
)

# Bouton pour exécuter l'action
button = widgets.Button(description="Send goal pose")

# Zone de sortie pour afficher les résultats
output = widgets.Output()

def on_slider_change(change):
    with output:
        clear_output()
        selected_index = change['new']
        selected_array = grasp_poses[selected_index]
        print(f"Tableau {selected_index + 1}:")
        print(selected_array)
        
        goto_id = arm.goto_from_matrix(selected_array)

        if goto_id != -1:
            while not manipulation_api.reachy.is_move_finished(goto_id):
                time.sleep(0.1)

            time.sleep(1.0)

            joints_pos = arm.get_joints_positions()
            fk = arm.forward_kinematics()
        
            l2_dist = compute_l2_distance(fk, selected_array)
            print(f'l2 dist: {l2_dist}')

        goto_id = symetrical_arm.goto_from_matrix(symetrical_grasp_poses[selected_index])
        if goto_id != -1:
            while not manipulation_api.reachy.is_move_finished(goto_id):
                time.sleep(0.1)

            time.sleep(1.0)
                                                  
# Associer la fonction de changement de valeur au slider
slider.observe(on_slider_change, names='value')

# Afficher les widgets
display(slider, output)

IntSlider(value=0, description='Tableau:', max=3)

Output()

In [32]:
manipulation_api.goto_rest_position(left=True)

In [30]:
manipulation_api.reachy.set_pose('zero')

GoToHomeId(head=id: 79
, r_arm=id: 80
, l_arm=id: 81
)